In [1]:
# Resnet 50

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roihist'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
model = ResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).to(device)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Using device: cpu
Checking directory: D:/project/with_roihist\0
Loaded image: D:/project/with_roihist\0\329637_series8_slice0_L_roi.png
Loaded image: D:/project/with_roihist\0\329637_series8_slice1_L_roi.png
Loaded image: D:/project/with_roihist\0\329637_series8_slice2_L_roi.png
Loaded image: D:/project/with_roihist\0\390116_series9_slice0_R_roi.png
Loaded image: D:/project/with_roihist\0\390116_series9_slice1_R_roi.png
Loaded image: D:/project/with_roihist\0\390116_series9_slice2_R_roi.png
Loaded image: D:/project/with_roihist\0\390116_series9_slice3_R_roi.png
Loaded image: D:/project/with_roihist\0\390116_series9_slice4_R_roi.png
Loaded image: D:/project/with_roihist\0\390116_series9_slice5_R_roi.png
Loaded image: D:/project/with_roihist\0\406320_series9_slice0_R_roi.png
Loaded image: D:/project/with_roihist\0\406320_series9_slice1_R_roi.png
Loaded image: D:/project/with_roihist\0\406320_series9_slice2_R_roi.png
Loaded image: D:/project/with_roihist\0\412857_series8_slice0_L_roi.png


KeyboardInterrupt: 

In [ ]:
# hybrid sampling

In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from collections import Counter
from torch.utils.data import Subset
from torch.utils.data import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter
import numpy as np
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L') 
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([  
    transforms.Resize((64, 64)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

def undersample_class(dataset, majority_class, target_count):
    majority_indices = [i for i, (_, label) in enumerate(dataset) if label == majority_class]
    undersampled_majority_indices = np.random.choice(majority_indices, size=target_count, replace=False)
    return undersampled_majority_indices

class OversampleDataset(Dataset):
    def __init__(self, dataset, labels, minority_classes):
        self.dataset = dataset
        self.labels = labels
        self.minority_classes = minority_classes
        
        self.minority_indices = [i for i, label in enumerate(self.labels) if label in self.minority_classes]
        self.majority_indices = [i for i, label in enumerate(self.labels) if label not in self.minority_classes]
        
        self.oversampled_minority_indices = random.choices(self.minority_indices, k=min(len(self.majority_indices), len(self.minority_indices) * 5))
        
        self.indices = self.majority_indices + self.oversampled_minority_indices
    
    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        actual_idx = self.indices[idx] % len(self.dataset)
        return self.dataset[actual_idx]

def hybrid_sampling(dataset, labels, majority_class, minority_classes):
    class_counts = Counter(labels)
    
    target_count = 500
    undersampled_majority_indices = undersample_class(dataset, majority_class, target_count)
    
    minority_indices = [i for i, (_, label) in enumerate(dataset) if label in minority_classes]
    
    combined_indices = list(undersampled_majority_indices) + minority_indices
    
    return Subset(dataset, combined_indices)

data_dir = 'D:/project/with_roi_histactual2'
dataset = CustomDataset(root_dir=data_dir, transform=transform)

labels = [label for _, label in dataset]  
class_counts = Counter(labels)

print("Class counts:", class_counts)

majority_class = max(class_counts, key=class_counts.get)
minority_classes = [cls for cls in class_counts if cls != majority_class]

print("Majority class:", majority_class)
print("Minority classes:", minority_classes)

train_size = int(0.85 * len(dataset))  
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataset = hybrid_sampling(train_dataset, labels, majority_class, minority_classes)

train_dataset = OversampleDataset(train_dataset, labels, minority_classes)



train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=32, 
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=32, 
                          shuffle=False)

print(f"Full dataset size: {len(dataset)}")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

model = ResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5)


def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device) 
            outputs = model(images).to(device)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) 
        optimizer.zero_grad()
        outputs = model(images).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    
    train_accuracy = calculate_accuracy(train_loader, model)
    test_accuracy = calculate_accuracy(test_loader, model)
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

KeyboardInterrupt: 

In [ ]:
# SMOTE

In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from imblearn.over_sampling import SMOTE

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

features = []
labels = []
for img, label in dataset:
    features.append(img.numpy().flatten())  # Flatten each image to a 1D array
    labels.append(label)

# Apply SMOTE to oversample minority classes
smote = SMOTE()
features_resampled, labels_resampled = smote.fit_resample(features, labels)
print(f"Class distribution after SMOTE: {Counter(labels_resampled)}")

# Convert resampled features and labels back to tensors
features_resampled = np.array(features_resampled).reshape(-1, 1, 64, 64)
features_tensor = torch.tensor(features_resampled, dtype=torch.float32)
labels_tensor = torch.tensor(labels_resampled, dtype=torch.long)

# Create a new dataset and split it
resampled_dataset = TensorDataset(features_tensor, labels_tensor)
train_size = int(0.85 * len(resampled_dataset))
test_size = len(resampled_dataset) - train_size
train_dataset, test_dataset = random_split(resampled_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
model = ResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).to(device)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

d:\pc_softwares\Python\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/100], Loss: 1.0902, Train Accuracy: 85.67%, Test Accuracy: 80.39%
Epoch [2/100], Loss: 0.4788, Train Accuracy: 88.35%, Test Accuracy: 81.83%
Epoch [3/100], Loss: 0.3981, Train Accuracy: 85.42%, Test Accuracy: 79.78%
Epoch [4/100], Loss: 0.4428, Train Accuracy: 85.33%, Test Accuracy: 82.55%
Epoch [5/100], Loss: 0.4141, Train Accuracy: 95.45%, Test Accuracy: 86.52%
Epoch [6/100], Loss: 0.2879, Train Accuracy: 86.48%, Test Accuracy: 76.53%
Epoch [7/100], Loss: 0.2124, Train Accuracy: 92.03%, Test Accuracy: 80.02%
Epoch [8/100], Loss: 0.2338, Train Accuracy: 89.24%, Test Accuracy: 81.59%
Epoch [9/100], Loss: 0.1974, Train Accuracy: 98.62%, Test Accuracy: 88.81%
Epoch [10/100], Loss: 0.1904, Train Accuracy: 97.96%, Test Accuracy: 88.45%
Epoch [11/100], Loss: 0.1961, Train Accuracy: 95.20%, Test Accuracy: 88.33%
Epoch [12/100], Loss: 0.1349, Train Accuracy: 93.58%, Test Accuracy: 85.92%
Epoch [13/100], Loss: 0.1316, Train Accuracy: 99.45%, Test Accuracy: 90.37%
Epoch [14/100], Loss:

KeyboardInterrupt: 

In [ ]:
# weight balancing with Inverse Number of Samples (INS) weights

In [14]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
class_counts = torch.zeros(3).to(device)  # Assuming 3 classes
for _, label in train_dataset:
    class_counts[label] += 1

# Calculate Inverse Number of Samples (INS) weights
weights = 1.0 / class_counts
weights = weights / weights.sum()  # Normalize weights

# Define the criterion with INS weights
model = ResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).to(device)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).to(device)
        loss = criterion(outputs, labels).to(device)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

d:\pc_softwares\Python\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/100], Loss: 2.2221, Train Accuracy: 60.25%, Test Accuracy: 55.23%
Epoch [2/100], Loss: 1.4066, Train Accuracy: 65.03%, Test Accuracy: 56.84%
Epoch [3/100], Loss: 1.0459, Train Accuracy: 53.48%, Test Accuracy: 38.87%
Epoch [4/100], Loss: 0.5954, Train Accuracy: 77.90%, Test Accuracy: 55.50%
Epoch [5/100], Loss: 0.7199, Train Accuracy: 51.21%, Test Accuracy: 31.37%
Epoch [6/100], Loss: 0.7878, Train Accuracy: 91.53%, Test Accuracy: 66.49%
Epoch [7/100], Loss: 0.8665, Train Accuracy: 71.56%, Test Accuracy: 46.92%
Epoch [8/100], Loss: 0.6736, Train Accuracy: 88.07%, Test Accuracy: 62.20%
Epoch [9/100], Loss: 1.0754, Train Accuracy: 75.44%, Test Accuracy: 52.55%
Epoch [10/100], Loss: 0.5680, Train Accuracy: 88.74%, Test Accuracy: 59.52%
Epoch [11/100], Loss: 0.5308, Train Accuracy: 98.63%, Test Accuracy: 68.36%
Epoch [12/100], Loss: 0.3909, Train Accuracy: 87.93%, Test Accuracy: 57.64%
Epoch [13/100], Loss: 0.2850, Train Accuracy: 94.79%, Test Accuracy: 69.17%
Epoch [14/100], Loss:

KeyboardInterrupt: 

In [ ]:
# weight balancing with Inverse of Square Root of Number of Samples (ISNS) weights

In [15]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
class_counts = torch.zeros(3)  # Assuming 3 classes
label_counts = Counter([label for _, label in train_dataset])
num_classes = len(label_counts)
class_weights = []

# Calculate weights based on the Inverse of Square Root of Number of Samples (ISNS)
for label in range(num_classes):
    if label_counts[label] > 0:
        class_weights.append(1.0 / np.sqrt(label_counts[label]))
    else:
        class_weights.append(0.0)  # Assign 0 weight if no samples in class

# Normalize weights to ensure they sum up to 1
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
class_weights = class_weights / class_weights.sum()

# Define the criterion with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
model = ResNet50(num_classes=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

In [ ]:
# weight balancing with Effective Number of Samples (ENS) weights

In [16]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def calculate_class_weights(labels, beta=0.99):
    """Calculates class weights using Effective Number of Samples (ENS) approach."""
    classes, counts = np.unique(labels, return_counts=True)
    effective_num = 1.0 - np.power(beta, counts)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * len(classes)  # Normalize weights
    return torch.tensor(weights, dtype=torch.float32).to(device)

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual3'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

train_labels = [label for _, label in train_dataset] 

class_weights = calculate_class_weights(train_labels)

criterion = nn.CrossEntropyLoss(weight=class_weights)
model = ResNet50(num_classes=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual3\0
Loaded image: D:/project/with_roi_histactual3\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0

In [ ]:
# offline data augmentation

In [18]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def apply_offline_augmentation(source_dir, dest_dir, num_augmentations=5):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    transform_augment = transforms.Compose([
        transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5)),  # Brightness & Contrast
        transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.5),  # Sharpness
        transforms.RandomHorizontalFlip(p=0.5),  # Flip
        transforms.RandomRotation(degrees=(-20, 20)),  # Rotation
        transforms.RandomResizedCrop(64, scale=(0.8, 1.2)),  # Zoom
    ])
    
    for label in os.listdir(source_dir):
        label_dir = os.path.join(source_dir, label)
        dest_label_dir = os.path.join(dest_dir, label)
        
        if not os.path.exists(dest_label_dir):
            os.makedirs(dest_label_dir)
        
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            image = Image.open(img_path).convert('L')
            
            # Save original image to the new directory
            image.save(os.path.join(dest_label_dir, img_name))
            
            # Apply augmentations and save
            for i in range(num_augmentations):
                augmented_image = transform_augment(image)
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.png"
                augmented_image.save(os.path.join(dest_label_dir, aug_img_name))

# Apply offline augmentation
source_dir = 'D:/project/with_roi_histactual2'
augmented_dir = 'D:/project/augmented_data2'
apply_offline_augmentation(source_dir, augmented_dir)

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
full_dataset = CustomDataset(root_dir=augmented_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
model = ResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Using device: cuda
Checking directory: D:/project/augmented_data2\0
Loaded image: D:/project/augmented_data2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/augmented_data2\0\329637_series8_slice0_L_roi_aug_0.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice0_L_roi_aug_1.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice0_L_roi_aug_2.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice0_L_roi_aug_3.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice0_L_roi_aug_4.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/augmented_data2\0\329637_series8_slice1_L_roi_aug_0.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice1_L_roi_aug_1.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice1_L_roi_aug_2.png
Loaded image: D:/project/augmented_data2\0\329637_series8_slice1_L_roi_aug_3.png
Loaded image: D:/project/augmented_data2\0\329637_ser

In [ ]:
# online data augmentation

In [19]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
        transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5)),  # Brightness & Contrast
        transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.5),  # Sharpness
        transforms.RandomHorizontalFlip(p=0.5),  # Flip
        transforms.RandomRotation(degrees=(-20, 20)),  # Rotation
        transforms.RandomResizedCrop(64, scale=(0.8, 1.2)),  # Zoom
        transforms.ToTensor(),  # Convert PIL image to tensor
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels * 4)
        )
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + identity
        return self.relu(out)

class ResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50, self).__init__()
        self.in_channels = 64

        # Stage 0
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stages
        self.layer1 = self._make_layer(64, 3, stride=1)   # 1 conv block + 2 identity blocks
        self.layer2 = self._make_layer(128, 4, stride=2)  # 1 conv block + 3 identity blocks
        self.layer3 = self._make_layer(256, 6, stride=2)  # 1 conv block + 5 identity blocks
        self.layer4 = self._make_layer(512, 3, stride=2)  # 1 conv block + 2 identity blocks

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self._initialize_weights()

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ConvBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(IdentityBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
model = ResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0